In [ ]:
import pandas as pd
import requests
import time

In [ ]:
# Read the CSV file containing SO (Sales Order) and request date
so_df = pd.read_csv('Adjust_SO_03072025_Production.csv')

api_url = 'http://10.116.2.72:8091/api/process-wo-request-dates'
api_key = None  # Set your API key if needed

batch_size = 10 
results = []

headers = {'Content-Type': 'application/json'}
if api_key:
    headers['Authorization'] = f'Bearer {api_key}'

for i in range(0, len(so_df), batch_size):
    batch = so_df.iloc[i:i+batch_size]
    so_updates = [
        {
            "so_number": str(row['SO']),
            "new_request_date": str(row['Request_date'])
        }
        for _, row in batch.iterrows()
    ]
    payload = {"so_updates": so_updates}
    print( payload)
    try:
        response = requests.post(api_url, json=payload, headers=headers)
        response.raise_for_status()
        resp_json = response.json()
        failed_wos = resp_json.get('failed_work_orders', [])
        # Optionally parse logs_csv for more details
        print(f"Batch {i//batch_size + 1} result: {resp_json}")
        if failed_wos:
            print(f"  Failed work orders in this batch: {failed_wos}")
        else:
            print("  All work orders in this batch succeeded.")
        results.append({'batch': i//batch_size + 1, 'status': 'success', 'response': resp_json})
    except Exception as e:
        results.append({'batch': i//batch_size + 1, 'status': f'error: {e}'})
        print(f"Batch {i//batch_size + 1} error: {e}")
    time.sleep(2.5)
